# Env

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234,
}
args = argparse.Namespace(**args)

print(args)

Namespace(seed=1234)


In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# Negative Log Likelihood

In [ ]:
# Generate linearly spaced vector
x = np.linspace(150, 190, 100)
print(x)

$P(x)={1 \over \sqrt{2 \pi \sigma^2}}e^{-{(x-\mu)^2 \over 2 \sigma^2}}$

In [ ]:
def gaussian_pdf(x, mean=0, std=1):
    """
    Gaussian PDF(Probability Density Function)
    :param x: 입력 값
    :return: gelu activation result
    """
    var = std * std
    p = 1 / np.sqrt(2 * np.pi * var) * np.exp(-np.square(x - mean) / (2 * var))
    return p

In [ ]:
# gaussian PDF
y = gaussian_pdf(x, mean=170, std=5)

# draw plot
plt.plot(x, y)
plt.title('Gaussian PDF')
plt.show()

In [ ]:
# sampled height
heights = [164.95, 165.35, 165.76, 166.16, 167.78, 168.99, 169.8, 170.2, 171.82, 172.63]
heights = np.array(heights)

In [ ]:
# Likelihood
lh = gaussian_pdf(heights, mean=170, std=5)

print(lh)
print(np.prod(lh))

In [ ]:
# mean value
means = np.arange(160, 181)
print(means)

In [ ]:
# 평균값별로 likelihood 계산
lhs = []
for mean in means:
    lh = gaussian_pdf(heights, mean=mean, std=5)
    lh = np.prod(lh)
    lhs.append(lh)
lhs = np.array(lhs)
max_idx = np.argmax(lhs)

print(max_idx, means[max_idx], lhs[max_idx])
print(lhs)

In [ ]:
# draw plot
plt.plot(means, lhs, '-o')
plt.title('Likelihood')
plt.show()

In [ ]:
# gaussian PDF
y = gaussian_pdf(x, mean=168, std=5)
lh = gaussian_pdf(heights, mean=168, std=5)

# draw plot
plt.plot(x, y)
plt.plot(heights, lh, "ro")
plt.title('Gaussian PDF')
plt.show()

In [ ]:
# 평균값별로 log likelihood 계산
lls = []
for mean in means:
    lh = gaussian_pdf(heights, mean=mean, std=5)
    ll = np.log(lh)
    ll = np.sum(ll)
    lls.append(ll)
lls = np.array(lls)
max_idx = np.argmax(lls)

print(max_idx, means[max_idx], lls[max_idx])
print(lls)

In [ ]:
# draw plot
plt.plot(means, lls, '-o')
plt.title('Log Likelihood')
plt.show()

In [ ]:
# 평균값별로 negative log likelihood 계산
nlls = -(lls)
min_idx = np.argmin(nlls)

print(min_idx, means[min_idx], nlls[min_idx])
print(nlls)

In [ ]:
# draw plot
plt.plot(means, nlls, '-o')
plt.title('Negative Log Likelihood')
plt.show()

# Example

In [ ]:
# Generate linearly spaced vector
p = np.linspace(0, 1, 102)[1:-1]
print(p)

$\mathcal{L}(p) = p^7(1-p)^3$

In [ ]:
# likelihood
lh = np.power(p, 7) * np.power(1 - p, 3)

In [ ]:
# draw plot
plt.plot(p, lh)
plt.title('Likelihood')
plt.show()

$\log \mathcal{L}(p) = 7 \log p + 3 \log (1-p)$

In [ ]:
# log likelihood
ll = 7 * np.log(p) + 3 * np.log(1 - p)

In [ ]:
# draw plot
plt.plot(p, ll)
plt.title('Log Likelihood')
plt.show()

$-\log \mathcal{L}(p) = - 7 \log p - 3 \log (1-p)$

In [ ]:
# negative log likelihood
nll = - 7 * np.log(p) - 3 * np.log(1 - p)

In [ ]:
# draw plot
plt.plot(p, nll)
plt.title('Negative Log Likelihood')
plt.show()

$- {\log \mathcal{L}(p) \over dp} = - 7 {1 \over p} + 3 {1 \over 1 -p}$

In [ ]:
# negative log likelihood gradient
gradient = - 7 / p + 3 / (1 - p)

In [ ]:
# draw plot
plt.plot(p, gradient)
plt.title('NLL Gradient')
plt.show()

# 실습
- np.random.rand 함수를 이용해 10개의 확률을 생성하세요.
- 생성된 확률의 likelihood를 구하세요.
- 생성된 확률의 log likelihood를 구하세요.
- 생성된 확률의 negative log likelihood를 구하세요.